In [1]:
#Imports
#%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    #DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pdpbox
    !pip install shap

# If you're working locally:
else:
    DATA_PATH = '../data/'

import datetime
import pandas as pd
import numpy as np

from category_encoders import OrdinalEncoder, OneHotEncoder

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
#Yes today (Regression)
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
#Not today (Classification)
#Tomorrow, both
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.inspection import permutation_importance
#import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor

import matplotlib.pyplot as plt

In [4]:
#Raw meterorological data
met_readings = pd.read_csv('BW_MET46251.txt', sep=' ', skipinitialspace=True)

#Basically government-made features made out of spectral data
directions = pd.read_csv('BW_SPEC46251.spec.txt', sep=' ', skipinitialspace=True)


def timeFixer(df):
    #first row is weird
    row1 = df.index[0]
    #drop weird first row
    df.drop(df.index[0],inplace=True)
    #add 00 for seconds
    df['ss'] = '00'
    #create date column
    df['Date'] = df['#YY']+'/'+df['MM']+'/'+df['DD']+' '+df['hh']+':'+df['mm']+':'+df['ss']
    #Convert df['Date'] to DateTime object
    #df['Date'] = datetime.datetime.strptime(df['Date'], '%Y/%m/%d %H:%M:%s.%f')
    df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d %H:%M:%S')
    #Index on date
    df.index = df['Date']
    #Drop now uneeded date & time columns
    df.drop(columns=['#YY','MM','DD','hh','mm','ss','Date'],inplace=True)
    #If more than 11 columns, drop extras
    extras = ['WDIR','WSPD','GST','PRES','ATMP','DEWP','VIS','PTDY','TIDE','MWD','WVHT','APD']
    if len(df.columns) > 11:
        df.drop(columns=extras, inplace=True)
    
    return df

met = timeFixer(met_readings)
directions = timeFixer(directions)

new_test1 = pd.read_csv('Test46251.txt', sep=' ', skipinitialspace=True)
new_test2 = pd.read_csv('test2.spec.txt', sep=' ', skipinitialspace=True)
nt1 = timeFixer(new_test1)
nt2 = timeFixer(new_test2)

display(met.head(1))
display(directions.head(1))

combined = pd.concat([met,directions],axis=1)
new_test = pd.concat([nt1,nt2],axis=1)

display(combined.head(2))
print('Combined shape:',combined.shape)
combined = combined.dropna()
new_test = new_test.dropna()
print('Combined shape minus rows with NaNs',combined.shape)

#Convert numericals to floats
numerical_cols = ['WVHT','DPD','WTMP','SwH','SwP','WWH','WWP','APD','MWD']
categorical_cols = ['SwD','WWD','STEEPNESS']
combined[numerical_cols] = combined[numerical_cols].astype('float')
new_test[numerical_cols] = new_test[numerical_cols].astype('float')

combined_comp = combined['STEEPNESS'].value_counts(normalize=True)
most_class = combined['STEEPNESS'].value_counts(normalize=True).max()
print('---')
print('Relative Frequency:',combined_comp)
print('---')
print('Proportion of most common class:',(most_class*100))
#The most common class is 'average' at 58%

#There are 45 days so 27/9/9 split
cutoff_train = '2021-04-01 00:00:00'
cutoff_val = '2021-04-10 00:00:00'
cutoff_test = '2021-04-20 00:00:00'
cutoff_new_test = '2021-4-20 00:00:00'

train = combined.loc[combined.index < cutoff_train]
combined = combined.loc[combined.index > cutoff_train]

val = combined.loc[combined.index < cutoff_val]
combined = combined.loc[combined.index > cutoff_val]

test = combined.loc[combined.index < cutoff_test]

test_new = new_test.loc[new_test.index>cutoff_test]

train_range = [train.index[1],train.index[-1]]
val_range = [val.index[1],val.index[-1]]
test_range = [test.index[1],test.index[-1]]
print('---')
print('Train:',train_range,'Length:',len(train))
print('Val:',val_range,'Length:',len(val))
print('Test:',test_range,'Length:',len(test))
print(len(test_new))

target = 'MWD'
X_train = train.drop(columns=target)
y_train = train[target]

X_val = val.drop(columns=target)
y_val = val[target]

X_test = test.drop(columns=target)
y_test = test[target]

X_new_test = new_test.drop(columns=target)
y_new_test = new_test[target]

print('---')
print('Train:',X_train.shape,y_train.shape,'Val:',X_val.shape,y_val.shape,'Test:',X_test.shape,y_test.shape)

print('---')
baseline_acc = y_train.mean()
y_pred = [y_train.mean()] * len(y_train)
print('Mean Wave Direction:',round(baseline_acc,2))
print('Mean Absolute Error of Naive Regressor:',mean_absolute_error(y_train,y_pred))



,DPD,WTMP
Date,,
2021-04-19 20:26:00,14,16.1


,WVHT,SwH,SwP,WWH,WWP,SwD,WWD,STEEPNESS,APD,MWD
Date,,,,,,,,,,
2021-04-19 20:26:00,1.3,0.7,14.3,1.1,6.7,SSW,W,AVERAGE,5.6,198


,DPD,WTMP,WVHT,SwH,SwP,WWH,WWP,SwD,WWD,STEEPNESS,APD,MWD
Date,,,,,,,,,,,,
2021-03-05 00:26:00,10,14.9,1.1,0.6,12.5,0.9,9.9,W,W,AVERAGE,7.7,276
2021-03-05 00:56:00,11,14.4,1.1,0.6,10.5,0.9,9.9,W,W,AVERAGE,7.5,264


Combined shape: (1786, 12)
Combined shape minus rows with NaNs (1744, 12)
---
Relative Frequency: AVERAGE       0.583142
VERY_STEEP    0.198968
SWELL         0.112385
STEEP         0.105505
Name: STEEPNESS, dtype: float64
---
Proportion of most common class: 58.31422018348624
---
Train: [Timestamp('2021-03-05 00:56:00'), Timestamp('2021-03-31 23:56:00')] Length: 856
Val: [Timestamp('2021-04-01 00:56:00'), Timestamp('2021-04-09 23:56:00')] Length: 419
Test: [Timestamp('2021-04-10 00:56:00'), Timestamp('2021-04-19 20:26:00')] Length: 469
158
---
Train: (856, 11) (856,) Val: (419, 11) (419,) Test: (469, 11) (469,)
---
Mean Wave Direction: 255.24
Mean Absolute Error of Naive Regressor: 26.141649925757708


In [5]:
#Adaptive Boosting Regressor
model_abr = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    AdaBoostRegressor(random_state=42)
)
model_abr.fit(X_train,y_train)

/Users/rowenwitt/.local/share/virtualenvs/testenv-_GQo1gTb/lib/python3.9/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['SwD', 'WWD', 'STEEPNESS'],
                               use_cat_names=True)),
                ('adaboostregressor', AdaBoostRegressor(random_state=42))])

In [11]:
def check_metrics(model):
    print('---')
    print(model)
    print('Training MAE:', mean_absolute_error(y_train, model.predict(X_train)))
    print('Validation MAE:', mean_absolute_error(y_val, model.predict(X_val)))
    print('Validation R^2:', model.score(X_val,y_val))
    print()
    print()


check_metrics(model_abr)

---
Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['SwD', 'WWD', 'STEEPNESS'],
                               use_cat_names=True)),
                ('adaboostregressor', AdaBoostRegressor(random_state=42))])
Training MAE: 4.937723735251001
Validation MAE: 5.8945584794076025
Validation R^2: 0.9549167976634525




In [12]:
#X_new test prediction of MWD
y_pred = model_abr.predict(X_new_test)
test_mae = mean_absolute_error(y_new_test,model_abr.predict(X_new_test))
test_R2 = model_abr.score(X_new_test,y_new_test)

print(test_mae,test_R2)

5.288878571039399 0.9614769957410216


In [ ]:
##Below is tidal stuff

In [176]:
#Working with tides
tides_dates_times = pd.read_csv('BW_MET46251.txt', sep=' ', skipinitialspace=True)
tides = pd.read_csv('9413745.txt',delim_whitespace=True)
test_times_X = X_test.copy()


tides_dates_times['Date'] = tides_dates_times['#YY']+'/'+tides_dates_times['MM']+'/'+tides_dates_times['DD']+' '+tides_dates_times['hh']+':'+tides_dates_times['mm']

#Now I need to convert both time formats into common datetime format expected by prophet, which I'm 
#Hoping to use to infill tidal values, though there is likely a fundamentally more sound way
#To accomplish this.  Tides need to convert to from 24 hour time to datetime.
#from datetime import datetime

tides['Time'] = tides['Day']+' '+tides['Time']
for i in tides.Time:
    tides['a'] = tides['Time'].str[:2].astype('int')
    tides['b'] = tides['Time'].str[3:5].astype('int')
    tides['c'] = tides['Time'].str[-2:]
    tides['d'] = tides['Time']
drop = ['Day','Time']
tides.drop(columns=drop,inplace=True)

a = tides.loc[tides['c']=='PM']
d = a.copy()
d['a'] = d['a']+12
b = tides.loc[tides['c']=='AM']
c = pd.concat([b,d])
#c['dt'] = c.index+' '+c['a'].astype('str').str[0:4]+':'+c['b'].astype('str')+':'+'00'
c['e'] = c.index+' '+c['d']
c['dt'] = pd.to_datetime(c['e'],infer_datetime_format=True)
c.index = c['dt']
drop=['a','b','c','d','dt','e']
c.drop(columns=drop,inplace=True)
c = c.sort_index()
c['time'] = c.index

c['time2'] = c['time'].shift(1)
c['time3'] = (c['time']-c['time2']).astype('timedelta64[h]')
c['Pred2'] = c['Pred'].shift(1)
c['per_dif'] = c['Pred']-c['Pred2']
c['del'] = c['per_dif'] / c['time3']



c

,Date,Pred,High/Low,time,time2,time3,Pred2,per_dif,del
dt,,,,,,,,,
2021-03-01 05:27:00,Mon,1.03,L,2021-03-01 05:27:00,NaT,NaN,NaN,NaN,NaN
2021-03-01 11:27:00,Mon,4.82,H,2021-03-01 11:27:00,2021-03-01 05:27:00,6.0,1.03,3.79,0.631667
2021-03-01 17:48:00,Mon,0.10,L,2021-03-01 17:48:00,2021-03-01 11:27:00,6.0,4.82,-4.72,-0.786667
2021-03-02 00:17:00,Tue,4.82,H,2021-03-02 00:17:00,2021-03-01 17:48:00,6.0,0.10,4.72,0.786667
2021-03-02 06:24:00,Tue,0.77,L,2021-03-02 06:24:00,2021-03-02 00:17:00,6.0,4.82,-4.05,-0.675000
...,...,...,...,...,...,...,...,...,...
2021-03-30 18:11:00,Tue,0.93,L,2021-03-30 18:11:00,2021-03-30 12:33:00,5.0,4.29,-3.36,-0.672000
2021-03-31 00:29:00,Wed,5.41,H,2021-03-31 00:29:00,2021-03-30 18:11:00,6.0,0.93,4.48,0.746667
2021-03-31 07:15:00,Wed,-0.45,L,2021-03-31 07:15:00,2021-03-31 00:29:00,6.0,5.41,-5.86,-0.976667


In [14]:
#To impute tidal data
t = []    
pp = []
p = []
for j in range(1,len(c)):    #For length of tide dataframe
    p = []
    for i in range(1,((c['time3'][j]).astype('int')-1)): #For the number of hours between observations j & j+1
        interval = datetime.timedelta(hours=i) #Add iterating hours
        t.append(c['time2'][j]+interval) #add to list of 6 datetimes
        p.append(abs(c['Pred'][j]+(c['del'][j]-(i)))) #Add prediction (prev pred + i * rate)
    p.reverse()
    pp.extend(p)
data = {'time':t,'Pred':pp}
mp = pd.DataFrame(data)
mp.index = mp['time']
mp.head(10)
cmp = pd.concat([mp,c])

#cmp.sort_index().head(5)

cmp_drop = ['Date','time2','time3','Pred2','per_dif','del']
cmp.drop(columns=cmp_drop,inplace=True)
cmp['day'] = cmp.index.day
cmp['time'] = cmp.index.hour
cmp = cmp.sort_values(['day','time'],ascending=[True,True])
cmp = cmp.reset_index()
cmp

,index,time,Pred,High/Low,day
0,2021-03-01 05:27:00,5,1.030000,L,1
1,2021-03-01 06:27:00,6,1.451667,NaN,1
2,2021-03-01 07:27:00,7,2.451667,NaN,1
3,2021-03-01 08:27:00,8,3.451667,NaN,1
4,2021-03-01 09:27:00,9,4.451667,NaN,1
...,...,...,...,...,...
564,2021-03-31 13:38:00,13,3.850000,H,31
565,2021-03-31 14:38:00,14,1.946000,NaN,31
566,2021-03-31 15:38:00,15,0.946000,NaN,31
567,2021-03-31 16:38:00,16,0.054000,NaN,31


In [15]:
c_drop = ['time']
c.drop(columns=c_drop,inplace=True)
c['day'] = c.index.day
c['time'] = c.index.hour
c = c.reset_index()
c
ccmp = pd.concat([cmp,c])
ccmp = ccmp.sort_values(['day','time'],ascending=[True,True])
ccmp.reset_index()

ccmp['key'] = (ccmp['time']).astype('str') + (ccmp['day']).astype('str')
ccmp.drop(columns=['High/Low','dt','Date','time2','time3','Pred2','per_dif','del'],inplace=True)
ccmp.dropna(inplace=True)
ccmp

,index,time,Pred,day,key
0,2021-03-01 05:27:00,5,1.030000,1,51
1,2021-03-01 06:27:00,6,1.451667,1,61
2,2021-03-01 07:27:00,7,2.451667,1,71
3,2021-03-01 08:27:00,8,3.451667,1,81
4,2021-03-01 09:27:00,9,4.451667,1,91
...,...,...,...,...,...
564,2021-03-31 13:38:00,13,3.850000,31,1331
565,2021-03-31 14:38:00,14,1.946000,31,1431
566,2021-03-31 15:38:00,15,0.946000,31,1531
567,2021-03-31 16:38:00,16,0.054000,31,1631


In [271]:
#I've imputed my tidal data, now I'll combine it into my df
#I'll just make a copy of the old one
X_train_tides = X_train.copy()
y_train_tides = y_train.copy()

X_train_tides['time'] = X_train_tides.index.hour
X_train_tides['day'] = X_train_tides.index.day
X_train_tides['key'] = (X_train_tides['time']).astype('str') + (X_train_tides['day']).astype('str')

X_train_tides_c = pd.merge(X_train_tides,ccmp,on='key',how='inner')
X_train_tides_c.drop_duplicates(inplace=True)
X_train_tides_c.index = X_train_tides_c['index']
X_train_tides_c.drop(columns=['time_x','day_x','key','index','time_y','day_y'],inplace=True)
print(X_train_tides_c.shape)


X_train_tides_c = X_train_tides_c.drop_duplicates(subset='Pred',keep='first')
X_train_tides_c.sort_index().head(5)

(714, 12)


,DPD,WTMP,WVHT,SwH,SwP,WWH,WWP,SwD,WWD,STEEPNESS,APD,Pred
index,,,,,,,,,,,,
2021-03-01 12:27:00,6.0,12.8,2.5,1.1,13.3,2.3,6.2,W,WNW,VERY_STEEP,6.0,4.686667
2021-03-02 12:25:00,6.0,12.9,2.2,0.7,11.8,2.1,6.2,W,WNW,VERY_STEEP,5.8,4.260000
2021-03-02 22:24:00,6.0,12.9,2.1,0.7,10.5,2.0,5.6,W,WNW,VERY_STEEP,5.7,4.738333
2021-03-03 22:02:00,6.0,13.1,2.2,0.6,16.7,2.2,5.6,SW,WNW,VERY_STEEP,5.5,3.768333
2021-03-04 12:41:00,6.0,12.8,2.0,0.6,15.4,2.0,6.2,S,W,VERY_STEEP,5.3,2.673333


In [254]:

X_test_tides = X_test.copy()
X_test_tides['dam'] = X_test_tides.index
X_test_tides['hour'] = X_test_tides['dam'].dt.hour
X_test_tides['day'] = X_test_tides['dam'].dt.day
X_test_tides['key'] = (X_test_tides['day']).astype('str') + (X_test_tides['hour']).astype('str')
X_test_tides = X_test_tides.reset_index()

tides_test_raw = pd.read_csv('24hr9413745.txt',delim_whitespace=True)
tides_test_raw['both'] = tides_test_raw['Date']+' '+tides_test_raw['Time']
tides_test_raw['both'] = pd.to_datetime(tides_test_raw['both'])
tides_test_raw['both2'] = tides_test_raw['both'].shift(1)
tides_test_raw['time3'] = tides_test_raw['both'] - tides_test_raw['both2']
tides_test_raw['time3'] = (pd.to_numeric(tides_test_raw['time3'].dt.seconds,downcast='integer')/3600)
tides_test_raw = tides_test_raw[1:]
tides_test_raw['time3'] = round(tides_test_raw['time3'],0).astype('int')
tides_test_raw['day'] = tides_test_raw['both'].dt.day
tides_test_raw['Pred2'] = tides_test_raw['Pred'].shift(1)
tides_test_raw['dif'] = tides_test_raw['Pred'] - tides_test_raw['Pred2']
tides_test_raw['vel'] = tides_test_raw['dif'] / tides_test_raw['time3']
tides_test_raw['day'] = tides_test_raw['both'].dt.day
tides_test_raw['hour'] = tides_test_raw['both'].dt.hour
tides_test_raw['key'] = tides_test_raw['day'].astype('str')+tides_test_raw['hour'].astype('str')
tides_test_raw


#To impute tidal data
t = []    
pp = []
p = []
for j in range(1,len(tides_test_raw)):    #For length of tide dataframe
    p = []
    for i in range(1,((tides_test_raw['time3'][j]).astype('int')-1)): #For the number of hours between observations j & j+1
        interval = datetime.timedelta(hours=i) #Add iterating hours
        t.append(tides_test_raw['both2'][j]+interval) #add to list of 6 datetimes
        p.append(abs(tides_test_raw['Pred'][j]+(tides_test_raw['vel'][j]-(i)))) #Add prediction (prev pred + i * rate)
    p.reverse()
    pp.extend(p)
data = {'time':t,'Pred':pp}
mp_test = pd.DataFrame(data) 
mp_test.index = mp_test['time']  #I know this worked
#I need to now combine these predictions with my test_set

mp_test
mp_test['day'] = mp_test.index.day
mp_test['hour'] = mp_test.index.hour
mp_test['key'] = mp_test['day'].astype('str')+mp_test['hour'].astype('str')
mp_test.drop(columns=['time','day','hour'],inplace=True)
mp_test = mp_test.reset_index()
mp_test.drop(columns='time',inplace=True)
mp_test

,Pred,key
0,NaN,105
1,NaN,106
2,NaN,107
3,NaN,108
4,4.181667,1011
...,...,...
160,5.497500,206
161,4.497500,207
162,3.497500,208
163,2.497500,209


In [288]:
test_tides = pd.merge(left=mp_test,right=X_test_tides,on='key')
test_tides = test_tides.drop_duplicates(subset='Pred',keep='first')
test_tides['Index'] = test_tides['Date']
holder = test_tides['Pred']
test_tides.index = test_tides['Index']
test_tides['Pred2'] = test_tides['Pred']
test_tides.drop(columns=['Date','key','dam','hour','day','Pred','Index'],inplace=True)
test_tides['Pred'] = test_tides['Pred2']
test_tides.drop(columns=['Pred2'],inplace=True)
test_tides

,DPD,WTMP,WVHT,SwH,SwP,WWH,WWP,SwD,WWD,STEEPNESS,APD,Pred
Index,,,,,,,,,,,,
2021-04-10 05:26:00,7.0,13.9,2.1,0.5,13.3,2.0,6.7,SW,W,STEEP,5.6,NaN
2021-04-10 11:26:00,6.0,14.0,1.9,0.4,11.1,1.9,5.9,W,WNW,VERY_STEEP,5.4,4.181667
2021-04-10 12:26:00,7.0,14.1,2.0,0.4,10.5,2.0,6.7,WNW,W,STEEP,5.3,3.181667
2021-04-10 13:26:00,6.0,14.1,2.1,0.4,13.3,2.1,5.9,S,WNW,VERY_STEEP,5.3,2.181667
2021-04-10 14:26:00,7.0,14.1,2.1,0.4,10.5,2.1,7.1,W,W,STEEP,5.4,1.181667
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-19 13:26:00,14.0,15.9,1.5,0.7,14.3,1.3,6.2,SSW,W,AVERAGE,5.7,1.293750
2021-04-19 14:26:00,15.0,15.9,1.4,0.7,15.4,1.1,5.6,SSW,WNW,AVERAGE,5.8,0.293750
2021-04-19 15:26:00,6.0,15.9,1.2,0.6,20.0,1.1,5.9,WSW,W,STEEP,5.8,0.706250


In [ ]:
#Now that I've finally fitted in my tidal data, I don't really want to do the same thing
#For the validation and test sets.  I could probably do them together (data leak?)
#What I should do is make an advanced wrangle function that does all the steps automatically
#It woudln't even take that long

#For now, I'm going to focus on some visualizations to satisfy the project constraints


In [289]:
X_train_tides_c

,DPD,WTMP,WVHT,SwH,SwP,WWH,WWP,SwD,WWD,STEEPNESS,APD,Pred
index,,,,,,,,,,,,
2021-03-05 02:32:00,10.0,13.8,1.1,0.6,10.5,0.9,9.9,W,W,AVERAGE,7.3,5.170000
2021-03-05 03:32:00,11.0,13.8,1.0,0.5,11.1,0.9,9.1,WSW,W,AVERAGE,6.6,5.612857
2021-03-05 04:32:00,13.0,13.8,1.0,0.6,12.5,0.9,9.9,WSW,W,AVERAGE,6.5,4.612857
2021-03-05 05:32:00,13.0,13.7,1.1,0.6,12.5,0.9,8.3,WSW,W,AVERAGE,6.3,3.612857
2021-03-05 06:32:00,11.0,13.7,1.0,0.6,10.5,0.8,9.9,W,W,AVERAGE,6.0,2.612857
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-31 13:38:00,15.0,13.3,0.9,0.5,15.4,0.7,7.7,SW,W,AVERAGE,7.0,3.850000
2021-03-31 14:38:00,17.0,13.2,1.0,0.6,16.7,0.8,9.1,SSW,W,AVERAGE,7.5,1.946000
2021-03-31 15:38:00,12.0,13.2,0.9,0.6,11.8,0.7,9.9,WNW,WNW,AVERAGE,7.1,0.946000


In [226]:

#mp_test.head(10)
#X_test_tides['time'] = X_test_tides.index
#X_test_tides = X_test_tides.reset_index()

#cmp = pd.merge(left=tides_test_raw,right=mp_test,on='key')

#cmp.sort_index().head(5)

# cmp_drop = ['Date','time2','time3','Pred2','per_dif','del']
# cmp.drop(columns=cmp_drop,inplace=True)
# cmp['day'] = cmp.index.day
# cmp['time'] = cmp.index.hour
# cmp = cmp.sort_values(['day','time'],ascending=[True,True])
# cmp = cmp.reset_index()

#cmp = cmp.drop_duplicates(subset='Pred',keep='first')



#tides_test_raw['time2'] = pd.to_datetime(tides_test_raw['time2'])
#tides_test_raw['time2'] = pd.to_datetime(tides_test_raw['time2'])
#tides_test_raw['time3'] = tides_test_raw['time']-tides_test_raw['time2']
# tides_test_raw['Pred2'] = tides_test_raw['Pred'].shift(1)
# #tides_test_raw['per_dif'] = tides_test_raw['Pred']-tides_test_raw['Pred2']
# #tides_test_raw['del'] = tides_test_raw['per_dif'] / tides_test_raw['time3']
# #tides_test_raw['time'] = datetime.timestamp(tides_test_raw['Time'])
# #tides_test_raw['time'] = tides_test_raw
#tides_test_raw['time'] = tides_test_raw['both'].dt.time
#tides_test_raw['time2'] = tides_test_raw['time'].shift(1)

